# Web scraping for most of the sitemap websites

In [20]:
# importing libraries

import csv 
import operator
import re
import threading
import random

import sys

import requests
from tqdm import tqdm 
from bs4 import BeautifulSoup

from concurrent.futures import ThreadPoolExecutor
import concurrent

from urllib.parse import urlparse
from urllib.parse import urljoin

import spacy # we use this for word similarity

from collections import defaultdict
import time


### Loading the links for scraping

In [21]:
links = []

with open('../link_data.csv', mode='r', newline='', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        links.append(row[0])

### Additional methods for storing data

In [22]:
def write_data_to_csv(data_to_write, csv_filename):
    # Open the file in append mode 'a' and ensure newline is handled correctly
    with open(csv_filename, "a", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Iterate over the list of tuples and append each to the CSV
        for row in data_to_write:
            csvwriter.writerow(row)

### Methods that will be used for scraping


In [ ]:
def get_base_url(url):
    try:
        parsed_url = urlparse(url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
        return base_url
    except Exception as e:
        # print(f"Error parsing URL {url}: {e}")
        return None


def clean_text(text):
    # Replace multiple spaces and line breaks with a single space
    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    return cleaned_text

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.1 Safari/605.1.15",
]

# List of proxy servers (this can be dynamically updated with proxy pools or services)
PROXIES = [
    {"http": "http://178.48.68.61:18080",  "https": "http://178.48.68.61:18080"},    
    # Add more proxies if available
]

# Function to get page content
def get_data(url):
    headers = {"User-Agent": random.choice(USER_AGENTS)}  # Rotate user-agent

    try:
        response = requests.get(url, headers=headers, timeout=3)

        # Handle rate-limiting (HTTP 429) by pausing and retrying
        if response.status_code == 429:
            print(f"Rate limit reached. Sleeping before retrying {url}")
            time.sleep(random.uniform(4, 8))  # Random delay to avoid detection
            return get_data(url)

        if response.status_code == 200:
            return response.content  # Return HTML content if successful

        print(f"Failed to retrieve {url}, Status Code: {response.status_code}")
        return None

    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None


def has_letters(input_string):
    return any(char.isalpha() for char in input_string)

def extract_data(url):
    html_data = get_data(url)
    if html_data is None:
        return None
    
    soup = BeautifulSoup(html_data, "html.parser")
    
    # Finding the title in the meta tags
    title = soup.find('title')
    if title:
        title = title.get_text()
    else: return None
        
    # Remove scripts, styles, and irrelevant content
    for script in soup(["script", "style", "footer", "nav", "header", "noscript", "head"]):
        script.extract()

    h1_tags = [clean_text(h1.get_text()) for h1 in soup.find_all('h1')]
    
    page_text = clean_text(soup.get_text(separator=' '))
    
    h1_tag = None
    # If no h1 tags are found, return None
    if h1_tags:
        h1_tag = h1_tags[0] 
 
    # We get the last path of the url
    url_index = url.rfind('/')
    url_last_path = url[url_index + 1:].replace('-', ' ')
    
    if not has_letters(url_last_path):
        url_last_path = None
    
    h1_tag_position = None
    if h1_tag in page_text:
        start_idx = page_text.index(h1_tag)
        end_idx = start_idx + len(h1_tag)
        h1_tag_position = (h1_tag, start_idx, end_idx)
    
    return h1_tag_position, title, url_last_path, page_text
    

final_data = []
links = links[:30000]  

final_data_lock = threading.Lock()  # A lock to ensure thread-safe appending to the list

# Function to process a single URL
def process_url(link):
    try:
        h1_tag, title, url_last_path, page_text = extract_data(link)
        with final_data_lock:  # Ensure thread-safe access to final_data
            final_data.append([link, h1_tag, title, url_last_path, page_text])
    except Exception as e:
        print(f"Error processing {link}: {e}")

# Read from CSV and process in parallel
with open('../dispersed_link_data.csv', mode='r', newline='', encoding='utf-8') as file:

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=32) as executor:  # Adjust max_workers as needed
        futures = []
        for link in links:
            futures.append(executor.submit(process_url, link))  # Submit tasks to the thread pool

        # Track progress with tqdm
        for _ in tqdm(concurrent.futures.as_completed(futures), total=len(links), desc="Processing URLs"):
            pass  # We don't need the result here since we're appending directly to final_data

                          


Processing URLs:   0%|          | 15/30000 [00:02<45:49, 10.90it/s]  

Error fetching https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds: HTTPSConnectionPool(host='www.hopewells.co.uk', port=443): Read timed out. (read timeout=3)
Error processing https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds: cannot unpack non-iterable NoneType object


Processing URLs:   0%|          | 22/30000 [00:03<30:21, 16.46it/s]

Error fetching https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds/somnus-excelsior-40000-springs: HTTPSConnectionPool(host='www.hopewells.co.uk', port=443): Read timed out. (read timeout=3)
Error processing https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds/somnus-excelsior-40000-springs: cannot unpack non-iterable NoneType object
Error fetching https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds/somnus-regent-8000-springs: HTTPSConnectionPool(host='www.hopewells.co.uk', port=443): Read timed out. (read timeout=3)
Error processing https://www.hopewells.co.uk/products/beds-bedroom-furniture/beds/somnus-regent-8000-springs: cannot unpack non-iterable NoneType object
Error fetching https://www.hopewells.co.uk/products/brands/ercol-fine-furniture-for-your-home/ercol-bosco-bedroom/ercol-1360-bosco-double-bed: HTTPSConnectionPool(host='www.hopewells.co.uk', port=443): Read timed out. (read timeout=3)
Error processing https://www.hopewells.co.uk/produc

Processing URLs:   0%|          | 36/30000 [00:04<49:53, 10.01it/s]

...

Processing URLs:  71%|███████▏  | 21428/30000 [15:12<05:38, 25.34it/s]

Failed to retrieve https://designkollective.com/stores/chic-home/products/87381, Status Code: 404
Error processing https://designkollective.com/stores/chic-home/products/87381: cannot unpack non-iterable NoneType object


Processing URLs:  71%|███████▏  | 21448/30000 [15:12<05:08, 27.69it/s]

Failed to retrieve https://designkollective.com/stores/chic-home/products/87408, Status Code: 404
Error processing https://designkollective.com/stores/chic-home/products/87408: cannot unpack non-iterable NoneType object


Processing URLs:  72%|███████▏  | 21671/30000 [15:22<06:00, 23.13it/s]

In [ ]:
with open('../data/preprocessed_data_from_all_sitemaps.csv', mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['URL', 'h1', 'title', 'url_last_path', 'page_text'])
    for row in final_data:
        csv_writer.writerow(row)
        

print("Processing completed!")